In [1]:
pip install spark-nlp==3.3.4

    100% |████████████████████████████████| 143kB 7.6MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [30]:
spark

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:44429)
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:44429)

In [3]:
pip install numpy pandas nltk scikit-learn

    100% |████████████████████████████████| 13.4MB 103kB/s eta 0:00:01
    100% |████████████████████████████████| 9.5MB 148kB/s eta 0:00:01
    100% |████████████████████████████████| 1.5MB 972kB/s eta 0:00:01
    100% |████████████████████████████████| 20.0MB 69kB/s  eta 0:00:01
    100% |████████████████████████████████| 512kB 3.0MB/s eta 0:00:01
    100% |████████████████████████████████| 675kB 2.1MB/s eta 0:00:01
    100% |████████████████████████████████| 307kB 4.5MB/s eta 0:00:01
    100% |████████████████████████████████| 81kB 11.8MB/s ta 0:00:01
    100% |████████████████████████████████| 25.9MB 51kB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


# Imports

In [4]:
from sparknlp.annotator import *
import pyspark.sql.functions as f
from pyspark.sql import Window
import pyspark.sql.types as t
from pyspark.ml.feature import Tokenizer as pysparkTokenizer, HashingTF, StopWordsRemover, CountVectorizer
from pyspark.ml import Pipeline
from sparknlp.base import DocumentAssembler, Finisher
from sparknlp.annotator import Tokenizer, Normalizer, LemmatizerModel, StopWordsCleaner, PerceptronModel, Chunker
from pyspark.ml.clustering import LDA
from nltk.corpus import stopwords
import pandas as pd
import nltk
import sparknlp
nltk.download('stopwords')

from pyspark.sql.functions import regexp_extract, col
from pyspark.sql.functions import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
print("Spark NLP version", sparknlp.version())

Spark NLP version 3.3.4


## Preprocess tweeter data

In [6]:
df_tweet = spark.read.option("tableName", "Tweets").format("dynamodb").load().select(f.col("text"))
df_tweet = df_tweet.withColumn("user", regexp_extract(col("text"), r"@([A-Za-z0-9]+)", 1))
df_tweet = (df_tweet.withColumn('text', regexp_replace('text', r"@[A-Za-z0-9]+", ''))
            .withColumn('text', regexp_replace('text', "RT[\s:]+", '')))
df_tweet.show()
f"Lenght: {df_tweet.count()}"

+--------------------+--------------+
|                text|          user|
+--------------------+--------------+
|May every single ...|  KamalaHarris|
| best president i...|   BarackObama|
|         ended trump|   BarackObama|
| can you make a l...|      JoeBiden|
|Happy New Year fr...| MichelleObama|
| It's definitely ...|       tedcruz|
|Student debt hold...|     SenWarren|
|Happy New Year fr...| MichelleObama|
|Happy New Year fr...| MichelleObama|
|Happy New Year fr...| MichelleObama|
| Wow photoshopped...|HillaryClinton|
|Happy New Year fr...| MichelleObama|
|Happy New Year fr...| MichelleObama|
|     1 more main ...|      HouseGOP|
|Happy New Year fr...| MichelleObama|
|Need affordable h...| SenatorDurbin|
| Fighting alongsi...|     SenWarren|
|_Pence: And Happy...|          Mike|
|_Pence: Happy New...|          Mike|
|Happy New Year fr...| MichelleObama|
+--------------------+--------------+
only showing top 20 rows



'Lenght: 460'

In [7]:
df_acc = spark.read.option("header","true").csv("../accounts.csv").select("party", "username").distinct()
df_acc.show()
f"Lenght: {df_acc.count()}"

+-----+---------------+
|party|       username|
+-----+---------------+
|    D|  SenWhitehouse|
|    D| SenCoonsOffice|
|    R|     ThomTillis|
|    D|    SenStabenow|
|    R|RoundsforSenate|
|    D|       timkaine|
|    D|    SenJackReed|
|    R|  JohnnyIsakson|
|    D|      SenBooker|
|    R|  SenJohnHoeven|
|    R|  TheBushCenter|
|    R|RepublicanStudy|
|    R|SenatorLankford|
|    D| SenJeffMerkley|
|    D| MurrayCampaign|
|    R|   SenTomCotton|
|    R|     TeamCornyn|
|    R|    RogerWicker|
|    D|      Bob_Casey|
|    R|    TomCottonAR|
+-----+---------------+
only showing top 20 rows



'Lenght: 198'

In [8]:
df_tweet = df_tweet.join(df_acc, df_tweet.user == df_acc.username).select('party', 'text')
df_tweet.show()

+-----+--------------------+
|party|                text|
+-----+--------------------+
|    D|May every single ...|
|    D| best president i...|
|    D|         ended trump|
|    D| can you make a l...|
|    D|Happy New Year fr...|
|    R| It's definitely ...|
|    D|Student debt hold...|
|    D|Happy New Year fr...|
|    D|Happy New Year fr...|
|    D|Happy New Year fr...|
|    D| Wow photoshopped...|
|    D|Happy New Year fr...|
|    D|Happy New Year fr...|
|    R|     1 more main ...|
|    D|Happy New Year fr...|
|    D|Need affordable h...|
|    D| Fighting alongsi...|
|    D|Happy New Year fr...|
|    D|Happy New Year fr...|
|    D| https://t.co/KhX...|
+-----+--------------------+
only showing top 20 rows



In [9]:
f"Lenght: {df_tweet.count()}"

'Lenght: 437'

## Preprocess reddit data

In [10]:
df_reddit = spark.read.option("tableName", "RedditPosts").format("dynamodb").load().select(f.col("submission_id").alias("text"), f.col("subreddit"))
df_reddit = df_reddit.withColumn("party", initcap(col('subreddit')).substr(1,1)).select("party", "text")
df_reddit.show()
f"Lenght: {df_reddit.count()}"

+-----+--------------------+
|party|                text|
+-----+--------------------+
|    D|Sen. Rand Paul ev...|
|    D|   Political Cartoon|
|    D|Two Kentucky hist...|
|    D|Biden administrat...|
|    D|Here Are The Comp...|
|    D|Capitol police of...|
|    D|Obama to deliver ...|
|    D|'Where is Ron DeS...|
|    D|Parseltongue AR G...|
|    D|Biden tells Putin...|
|    D|      Thanks O'Biden|
|    D|Marjorie Taylor G...|
|    D|Statewide electio...|
|    D|Orange Mayor Demi...|
|    D|House Democrats A...|
|    D|Five Out Of Six A...|
|    D|Colorado governor...|
|    D|Colorado governor...|
|    D|A Lavish Tax Dodg...|
|    D|Governor Ralph No...|
+-----+--------------------+
only showing top 20 rows



'Lenght: 79'

## Join both sources

In [11]:
df = df_tweet#.limit(1000)#.union(df_reddit)
df.show()
f"Lenght: {df.count()}"

+-----+--------------------+
|party|                text|
+-----+--------------------+
|    D|May every single ...|
|    D| best president i...|
|    D|         ended trump|
|    D| can you make a l...|
|    D|Happy New Year fr...|
|    R| It's definitely ...|
|    D|Student debt hold...|
|    D|Happy New Year fr...|
|    D|Happy New Year fr...|
|    D|Happy New Year fr...|
|    D| Wow photoshopped...|
|    D|Happy New Year fr...|
|    D|Happy New Year fr...|
|    R|     1 more main ...|
|    D|Happy New Year fr...|
|    D|Need affordable h...|
|    D| Fighting alongsi...|
|    D|Happy New Year fr...|
|    D|Happy New Year fr...|
|    D| https://t.co/KhX...|
+-----+--------------------+
only showing top 20 rows



'Lenght: 437'

## Train test dataset

In [12]:
num_r = df.filter(df.party == 'R').count()
num_d = df.filter(df.party == 'D').count()

num_min = num_r if num_r < num_d else num_d
party_min = 'R' if num_r < num_d else 'D'

dff = df.filter(df.party == 'R').limit(num_min).union(
        df.filter(df.party == 'D').limit(num_min)
    )

dff.groupBy("party")\
    .count()\
    .withColumn("share", col('count')/dff.count())\
    .show()

+-----+-----+-----+
|party|count|share|
+-----+-----+-----+
|    D|  122|  0.5|
|    R|  122|  0.5|
+-----+-----+-----+



In [19]:
train, test = dff.randomSplit([0.7, 0.3], seed=42)

In [20]:
train.groupBy("party")\
    .count()\
    .withColumn("share", col('count')/train.count())\
    .show()

+-----+-----+------------------+
|party|count|             share|
+-----+-----+------------------+
|    D|   86|0.5119047619047619|
|    R|   82|0.4880952380952381|
+-----+-----+------------------+



In [21]:
test.groupBy("party")\
    .count()\
    .withColumn("share", col('count')/test.count())\
    .show()

+-----+-----+-------------------+
|party|count|              share|
+-----+-----+-------------------+
|    D|   36|0.47368421052631576|
|    R|   40| 0.5263157894736842|
+-----+-----+-------------------+



In [22]:
# actual content is inside description column
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

In [23]:
# embeddings = BertSentenceEmbeddings.pretrained("sent_small_bert_L2_128") \
#       .setInputCols("document") \
#       .setOutputCol("sentence_embeddings")

In [24]:
embeddings = BertSentenceEmbeddings.pretrained("sent_small_bert_L8_512") \
      .setInputCols("document") \
      .setOutputCol("sentence_embeddings")

sent_small_bert_L8_512 download started this may take some time.
Approximate size to download 149.1 MB
[OK!]


In [25]:
# the classes/labels/categories are in category column
classsifierdl = ClassifierDLApproach()\
    .setInputCols (["sentence_embeddings"])\
    .setOutputCol("class")\
    .setLabelColumn("party")\
    .setMaxEpochs (20)\
    .setEnableOutputLogs (True) \
    .setLr(5*10**(-3)) \
    .setDropout(0.5)

In [26]:
bert_sent_clf_pipeline = Pipeline(
    stages = [
        document,
        embeddings,
        classsifierdl
    ])

In [27]:
bert_Model = bert_sent_clf_pipeline.fit(train)

In [28]:
preds = bert_Model.transform(test)

In [29]:
pandpred = preds.select("party","class.result").toPandas()
pandpred.result = pandpred.result.apply(lambda x: x[0])

--- Logging error ---
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:44429)
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-29-1eca9cf3100f>", line 1, in <module>
    pandpred = preds.select("party","class.result").toPandas()
  File "/opt/spark/python/pyspark/sql/pandas/conversion.py", line 141, in toPandas
    pdf = pd.DataFrame.from_records(self.collect(), columns=self.columns)
  File "/opt/spark/python/pyspark/sql/dataframe.py", line 677, in collect
    sock_info = self._jdf.collectToPython()
  File "/opt/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1305, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/opt/spark/python/pyspark/sql/utils.py", line 111, in deco
    return f(*a, **kw)
  File "/opt/spark/python/lib/py4j-

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:44429)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
print(classification_report(pandpred.party, pandpred.result))

In [ ]:
print(accuracy_score(pandpred.party, pandpred.result))

In [ ]:
print(confusion_matrix(pandpred.party, pandpred.result))